# Chapter 7. 통계학과 머신러닝

## 7.4 선형모델과 신경망

* 신경망의 기본적인 구조를 소개함

* 파이썬으로 구현을 하면서 선형모델과 복잡한 머신러닝을 비교

### 7.4.1 이 절에서 다루는 예제

* iris 데이터 분류

### 7.4.2 용어 비교

* 통계모델과 신경망 머신러닝에서 사용하는 용어 비교

| 통계모델 | 머신러닝(신경망)|
|:--------:|:---------------:|
|독립변수  | 입력 벡터       | 
|종속변수  | 목표 벡터       |
|계수      | 가중치          |
|절편      | 편향            |


### 7.4.3 단순 퍼셉트론

* 싸이킷런의 함수를 이용해 간단한 신경망을 구성

    - 복잡한 모델은 Tensorflow나 PyTorch 같은 프레임워크를 사용

In [1]:
# 수치계산에 사용되는 라이브러리
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# 그래프를 그리기 위한 라이브러리
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# 통계모델을 추정하는데 사용하는 라이브러리
import statsmodels.formula.api as smf
import statsmodels.api as sm

# 다층 퍼셉트론용 애플리케이션
from sklearn.neural_network import MLPClassifier

# 샘플 데이터 불러오기
from sklearn.datasets import load_iris

# 테스트 데이터와 훈련 데이터 분리
from sklearn.model_selection import train_test_split

# 데이터 표준화용 라이브러리
from sklearn.preprocessing import StandardScaler

# 표시 자릿수 지정
%precision 3

# 그래프를 주피터 노트북에 그리기 위한 설정
%matplotlib inline

### 7.4.4 데이터 읽기와 다듬기 

In [2]:
iris = load_iris()

In [3]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [4]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [5]:
# 일부 데이터만 추출
X = iris.data[50:150, 0:2]
y = iris.target[50:150]

print("독립변수 행 수, 열 수 : ", X.shape)
print("종속변수 행 수, 열 수 : ", y.shape)

독립변수 행 수, 열 수 :  (100, 2)
종속변수 행 수, 열 수 :  (100,)


In [6]:
# train 75%, test 25% 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

print("독립변수 행 수, 열 수 : ", X_train.shape)
print("종속변수 행 수, 열 수 : ", y_train.shape)

독립변수 행 수, 열 수 :  (75, 2)
종속변수 행 수, 열 수 :  (75,)


### 7.4.5 로지스틱 회귀

* 신경망과 비교를 위해 로지스틱 회귀를 이용한 분석 실시

In [7]:
# 종속변수 확인

y_train[0:10]

array([1, 1, 2, 2, 2, 2, 1, 1, 1, 1])

* 1과 2를 이용해서 서로 다른 종을 나타내고 있음. 로지스틱 회귀를 이용해서 1과 2중 어떤 종에 속하는지 자동으로 분류토록 함

In [8]:
# 데이터 정리
# 독립변수 데이터프레임
X_train_df = pd.DataFrame(X_train, columns = ["sepal_len" , "sepal_wid"])

# 종속변수 데이터프레임
y_train_df = pd.DataFrame({"species": y_train - 1})

# 데이터프레임 결합
iris_train_df = pd.concat([y_train_df, X_train_df], axis = 1)

# 결과 출력
print(iris_train_df.head(3))

   species  sepal_len  sepal_wid
0        0        5.7        2.8
1        0        6.6        3.0
2        1        6.1        3.0


* 로지스틱 모델링

In [9]:
# 모든 변수를 다 넣은 모델

logi_mod_full = smf.glm("species ~ sepal_len + sepal_wid", data = iris_train_df, family = sm.families.Binomial()).fit()

In [10]:
# 길이만 넣은 모델 

logi_mod_len = smf.glm("species ~ sepal_len", data = iris_train_df, family = sm.families.Binomial()).fit()

In [11]:
# 폭만 넣은 모델 

logi_mod_wid = smf.glm("species ~ sepal_wid", data = iris_train_df, family = sm.families.Binomial()).fit()

In [12]:
# Null 모델 

logi_mod_null = smf.glm("species ~ 1", data = iris_train_df, family = sm.families.Binomial()).fit()

In [13]:
# AIC 비교

print("full model" , logi_mod_full.aic.round(3))
print("len model" , logi_mod_len.aic.round(3))
print("wid model" , logi_mod_wid.aic.round(3))
print("null model" , logi_mod_null.aic.round(3))

full model 76.813
len model 76.234
wid model 92.768
null model 105.318


* 길이만 이용한 모델이 폭과 길이 모두 이요한 모델보다 AIC가 낮게 나옴

* 그러므로 독립변수는 꽃받침 길이 1개만 사용한 logi_mod_len을 사용

In [14]:
logi_mod_len.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-16.4152,4.000,-4.104,0.000,-24.256,-8.575
sepal_len,2.6478,0.639,4.142,0.000,1.395,3.901


* 추정된 계수를 보면 꽃받침의 길이는 양의 계수를 가지고 있고, 길이가 길수록 2번 종일 가능성이 크다는 것을 알 수 있음

* 훈련 데이터와 테스트 데이터의 적중률 확인

In [15]:
# 데이터 정리
X_test_df = pd.DataFrame(X_test, columns = ["sepal_len", "sepal_wid"])

# 피팅 예측
logi_fit = logi_mod_len.fittedvalues.round(0)
logi_pred = logi_mod_len.predict(X_test_df).round(0)

# 정답수
true_train = np.sum(logi_fit == (y_train - 1))
true_test = np.sum(logi_pred == (y_test -1))

In [16]:
# 적중률 계산
result_train = true_train / len(y_train)
result_test = true_test / len(y_test)

In [17]:
# 결과 출력
print("훈련 데이터 적중률   : ", result_train)
print("테스트 데이터 적중률 : ", result_test)

훈련 데이터 적중률   :  0.7466666666666667
테스트 데이터 적중률 :  0.68


* 훈련 데이터와 테스트 데이터의 적중률(Accuracy)은 약 70% 전후로 나타남

### 7.4.6 표준화

* 신경망을 이용하려면 리지 회귀와 마찬가지로 독립변수를 표준화해야 함

    - sklearn.preprocessing의 StandardScaler 함수 사용

In [18]:
# 표준화를 위한 준비
scaler = StandardScaler()
scaler.fit(X_train)

# 표준화 실시
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

* 표준화 실시 시 scaler.fit(X_train)의 결과를 훈련 데이터와 테스트 데이터에 적용함

    - 같은 평균, 분산(표준편차)를 사용함


* 훈련 데이터의 독립변수의 표준편차가 1인지 확인함

* 훈련 데이터의 표준편차가 1로 변환되었다고 해서 테스트 데이터의 표준 편차도 1이 된다고 할 수는 없으나 훈련 데이터와 테스트 데이터에 같은 기준을 적용하는 것이 중요함

In [19]:
np.std(X_train_scaled, axis = 0)

array([1., 1.])

In [20]:
np.std(X_test_scaled, axis = 0)

array([0.74 , 0.679])

### 7.4.7 신경망

In [21]:
# 신경망 모델링 및 정확도 산출

nnet = MLPClassifier(hidden_layer_sizes = (100, 100),
                     alpha = 0.07,
                     max_iter = 10000,
                     random_state = 0)
nnet.fit(X_train_scaled , y_train)

MLPClassifier(alpha=0.07, hidden_layer_sizes=(100, 100), max_iter=10000,
              random_state=0)

In [22]:
# 정확도
print("훈련 데이터 적중률 : " , nnet.score(X_train_scaled, y_train))
print("테스트 데이터 적중률 : ", nnet.score(X_test_scaled, y_test))

훈련 데이터 적중률 :  0.8933333333333333
테스트 데이터 적중률 :  0.68


* MLPClassifier 함수를 이용 간단한 신경망 모형을 구축해 봄 

    - hidden_layer_size = (100, 100) 이라고 지정해서 은닉층을 2층으로 하여 각각의 은닉층에 100개의 유닛을 배치함
    
    - alpha = 0.07로 하여 정규화의 강도를 지정함
    
    - max_iter = 10000으로 지정하여 파라미터 추정을 위한 반복을 몇 번이나 할 지 설정함
    
    - random_state = 0 으로 난수를 고정함 
    
    - 활성화 함수는 ReLU, Optimization은 Adam (디폴트 값 이용)

* 학습 데이터의 피팅 정확도는 증가하였지만 테스트 데이터에 대한 정확도는 로지스틱 회귀 모델과 동일함

### 7.4.8 선형모델의 장점과 신경망의 장점

* 신경망 모델의 구조를 배울 때, '신경망 모델은 선형모델을 보다 복잡하게 확장한 것'이라고 설명

    - 선형모델로는 표현할 수 없는 복잡한 데이터도 신경망이라면 잘 모델링 할 수 있을 가능성이 커짐 -> 그래서 딥러닝의 시대가 열림


* 주의할 점은 어느 한 쪽이 우수하다고 하기 보다는 데이터나 목적에 따라서 적합한 분석 방법을 정하는 것이 중요

* 각 모델의 기본이 되는 이론을 배우는 것이 장단점을 외워서 적용하는 방법보다 지름길이 될 수 있음

## 7.5 Next Step

* 이 책의 저자는 독자가 앞으로 어떤 공부를 하면 좋을지 대략적인 방침을 제시해 줌

### 7.5.1 수리적인 측면 배우기

* 본 책에서는 파라미터 추정을 위한 알고리즘 등은 과감하게 그 내용을 생략함

    - 앞으로 복잡한  내용을 이해하기 위해서는 수식으로 된 표현을 읽어낼 수 있는 기술이 필요함


* 집합 -> 미적분 -> 선형대수 순으로 제시함 

### 7.5.2 고전 통계학 입문 배우기

* 고전적인 통계학 입문서에서는 대략 아래 내용을 배우게 됨

    1. 표본과 모집단, 확률변수와 확률분포의 관계
    
    2. 기술통계
    
    3. 신뢰구간
    
    4. 통계적가설검정

### 7.5.3 통계모델 배우기

* 일반선형모델은 현대 통계학의 첫걸음이 됨

* 이와 함께 베이즈 통계, 시계열 분석으로 확장

### 7.5.4 머신러닝 배우기

* 머신러닝 특히 지도학습을 배울 때는 예측 정확도를 높이기 위해 다양한 방법을 배움

    - 정규화, 데이터 전처리, 독립변수 차원 축소, 변수 간 조합을 통한 파생변수 생성 등

### 7.5.5 모델 평가 방법 배우기

* 머신 러닝은 심층학습, 그래디언트 부스팅, 서포트 벡터 머신 등 여러 모델이 있음 

    - 이들 모델 간의 성능 평가 방법을 배워야 비교하여 적절한 모델을 선택할 수 있음
    
    - 머신러닝에서는 주로 일반화 오차의 추정과 평가가 주요 작업이 됨
    
    - AIC 등의 편리한 기준이 없는 경우가 보통이며, 교차 검증법 등이 자주 이용됨

### 7.5.6 데이터 과학

* 데이터 과학이란 데이터의 취득과 관리 그리고 의사결정에 활용하는 등 그 내용은 다방면에 걸쳐 있음

* 이 책은 파이썬을 이용한 데이터 과학의 가장 기초적인 이론을 다루는 것을 목적으로 기획되었으며, 기초를 이해하는 것이 더 어려울지도 모르나 기초가 튼튼하면 바뀌는 라이브러리에 잘 적응하고 대응할 수 있음

* **기초 이론은 배신 당하지 않고 오래 사용할 수 있는 도구**라는 점을 강조함

# **소감**

파이썬을 활용하여 전반적인 통계학 부분을 정리해 볼 수 있는 좋은 책이다. 

데이터 분석을 얕게 해본 사람들은 통계학이 다루는 부분의 중요성을 간과하는 편인데, 저자는 그런 점에 주의를 기울이며 기초적인 내용을 차근히 잘 설명해 주어 좋았다.

이 책을 정리하고 나니 예전에 PPT로 그려놓았던 '데이터 과학자의 역할'에 대한 장표가 생각났다.

넓은 스펙트럼을 담당하고 수행해야 하는 데이터 과학자가 오랜만에 통계학에 대해 정리해 보고 싶을 때, 참 좋을 책이라 생각된다.

![image2.png](drawings/data_science1.png)

![image2.png](drawings/data_science2.png)